In [71]:
date = "2024-01-03"  # to get csv

In [72]:
import importlib
from pathlib import Path

from modules import module
import numpy as np
import pandas as pd

try:
    importlib.reload(module)  # reload module
except NameError:
    pass

# Load DF

In [73]:
path = Path(f"input/{date}.csv")
df = pd.read_csv(path)

target_cols = [
    "External ID",
    "Display Name",
    "Date of Birth",
    "Email",
    "Phone",
    "Contract Type",
    "From",
    "To",
    "Fully Paid Date",
    "Membership/Display Name",
    "Membership Status",
    "Partner/Branch/Display Name",
    "Partner/Date of Birth",
    "Partner/Age",
    "Partner/City",
    "Partner/Gender",
    "Partner/Household income/month",
    "Partner/Industry/Display Name",
    "Partner/Job Position",
    "Partner/Occupation",
    "Partner/Street",
    "Partner/Street2",
    "Followers (Partners)/Hobby",
    "Followers (Partners)/Interest",
    "Partner/Hobby",
    "Partner/Interest",
]
current_cols = list(df.columns)
map_cols = dict(zip(current_cols, target_cols))
df.columns = df.columns.map(map_cols)

print(df.shape)
df.sample(3)

(29338, 26)


,External ID,Display Name,Date of Birth,Email,Phone,Contract Type,From,To,Fully Paid Date,Membership/Display Name,...,Partner/Household income/month,Partner/Industry/Display Name,Partner/Job Position,Partner/Occupation,Partner/Street,Partner/Street2,Followers (Partners)/Hobby,Followers (Partners)/Interest,Partner/Hobby,Partner/Interest
23404,__export__.membership_membership_line_179736_d...,MUHAMMAD FAKHRI RAMADHAN PULUNGAN,NaN,NaN,NaN,NaN,2016-11-25,2018-07-25,NaN,NaN,...,NaN,NaN,NaN,NaN,"JALAN KRISAN 1 BLOK F-3/15, BSD",NaN,NaN,NaN,NaN,NaN
10716,__export__.membership_membership_line_191328_c...,INA MEDYANA,1993-05-21,inamedyana@gmail.com,082221993251,Private,2021-02-14,2022-02-13,2021-02-12,[GO.DLX.12] WSE GO 12 MONTH - KK,...,Below Rp 10.000.000,NaN,HEAD OPERATION,CREDIT,JL PERMATA NO 4 003/005,NaN,NaN,NaN,TRAVELING,"TRAVELING, FASHION"
28708,__export__.membership_membership_line_148793_2...,NATALIA ENDAH UTAMI,NaN,NaN,NaN,NaN,2015-07-22,2016-07-22,NaN,NaN,...,NaN,NaN,NaN,NaN,CIMANGGIS 006/002 CIPAYUNG CIPUTAT,NaN,NaN,NaN,NaN,NaN


# Clean DF

In [74]:
df_clean = (
    df
    # drop na
    .dropna(subset="From")
    .dropna(subset="To")
    # drop date too old
    .loc[lambda df_: pd.to_datetime(df_["From"]).dt.year >= 2020]
    # drop membership code na
    .loc[lambda df_: ~(df_["Membership/Display Name"].isna())]
    # filter cancelled member and free member and non member
    .loc[
        lambda df_: ~(
            df_["Membership Status"].isin(
                ["Cancelled Member", "Free Member", "Non Member"]
            )
        )
    ]
    # filter staff
    .loc[lambda df_: df_["Contract Type"] != "Employee"]
    # rename column
    .rename(
        columns=lambda c: (
            c.lower()
            .replace("/display name", "")
            .replace("/month", "")
            .replace("position", "")
            .strip()
            .replace(" ", "_")
            .replace("/", "_")
        )
    )
    .rename(columns=module.to_rename)
    # drop unused cols
    .drop(columns=module.to_drop)
    .assign(
        # map center name
        center=lambda df_: df_["center"].str.upper().map(module.center_map),
        # clean city
        city=lambda df_: (df_["city"].str.title().str.strip()),
        # clean dob
        dob=lambda df_: np.where(
            df_["dob"].isna(), pd.to_datetime(df_["dob2"]), pd.to_datetime(df_["dob"])
        ),
        # clean start_date, end_date, fp_date
        start_date=lambda df_: pd.to_datetime(df_["start_date"]),
        end_date=lambda df_: pd.to_datetime(df_["end_date"]),
        fp_date=lambda df_: pd.to_datetime(df_["fp_date"]),
        fp_month=lambda df_: df_["fp_date"].dt.strftime("%m / %b %Y"),
        fp_year=lambda df_: df_["fp_date"].dt.strftime("%Y"),
        # clean income
        income=lambda df_: df_["income"].astype(module.income_cat),
        # clean job
        job=lambda df_: np.where(
            df_["job1"].isna(),
            df_["job2"].str.lower().str.replace("[^\w\s]", "", regex=True).str.strip(),
            df_["job1"].str.lower().str.replace("[^\w\s]", "", regex=True).str.strip(),
        ),
        # create age
        age=lambda df_: (
            (df_["start_date"] - df_["dob"])
            .div(pd.Timedelta("365 days"))
            .apply(np.floor)
        ),
        # create id
        id=lambda df_: df_["name"] + " " + df_["dob"].astype(str),
        # create membership code
        membership_code=lambda df_: module.get_membership_code(df_["product"]),
        # create membership duration
        membership_duration=lambda df_: module.get_membership_duration(df_["product"]),
    )
    # merge with membership mapping to obtain membership
    .merge(
        right=pd.read_excel(Path("input/membership_mapping.xlsx")),
        on="membership_code",
        how="left",
    )
    # make sure that all corporate is mapped
    .assign(
        is_cpt=lambda df_: np.where(
            df_["product"].str.lower().str.contains("cpt|corporate|corp", regex=True),
            True,
            df_["is_cpt"],
        )
    )
    # sort by id
    .sort_values(["id", "end_date"], ascending=[True, False])
    # create is next contract col
    .assign(is_next_contract=lambda df_: df_["id"].duplicated(keep="last"))
    # drop unused columns
    .drop(columns=["dob2", "job1", "job2", "name"])
    # drop membership code na
    .loc[lambda df_: ~(df_["membership_code"].isna())]
    # sort column
    .sort_index(axis=1)
    # # ! create is_active
    .assign(
        active_jan_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 jan 2023"
        ),
        active_feb_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 feb 2023"
        ),
        active_mar_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 mar 2023"
        ),
        active_apr_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 apr 2023"
        ),
        active_may_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 may 2023"
        ),
        active_jun_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 jun 2023"
        ),
        active_jul_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 jul 2023"
        ),
        active_aug_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 aug 2023"
        ),
        active_sep_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 sep 2023"
        ),
        active_oct_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 oct 2023"
        ),
        active_nov_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 nov 2023"
        ),
        active_dec_2023=lambda df_: module.is_active(
            df_, "start_date", "end_date", "1 dec 2023"
        ),
    )
)
df_clean

,addon_1,addon_2,addon_3,age,center,city,contract_type,core_product,dob,email,...,active_mar_2023,active_apr_2023,active_may_2023,active_jun_2023,active_jul_2023,active_aug_2023,active_sep_2023,active_oct_2023,active_nov_2023,active_dec_2023
8757,NaN,NaN,NaN,26.0,KK,Kota Makassar,Private,Go,1994-11-27,achmadjailani027@gmail.com,...,False,False,False,False,False,False,False,False,False,False
7683,NaN,NaN,NaN,30.0,LW,Jakarta Timur,Private,Deluxe,1990-09-10,andichintyaseptiani@gmail.com,...,False,False,False,False,False,False,False,False,False,False
4929,NaN,NaN,NaN,36.0,KK,Kab. Gowa,Private,Deluxe,1986-05-21,yunisas.manok@gmail.com,...,True,True,True,True,True,True,True,True,True,False
9533,NaN,NaN,NaN,34.0,KK,Kab. Gowa,Private,Deluxe,1986-05-21,yunisas.manok@gmail.com,...,False,False,False,False,False,False,False,False,False,False
1275,IELTS,NaN,NaN,33.0,NST,Rembang,Private,Go,1990-04-24,fadly.fajar90@gmail.com,...,False,False,False,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349,NaN,NaN,NaN,32.0,NST,Bekasi,Private,Deluxe,1990-05-05,znadziroh@gmail.com,...,True,True,True,True,True,True,True,True,True,True
11560,Proskill,NaN,NaN,19.0,PP,Bekasi,Private,Go,2001-04-27,zusmarrie27@gmail.com,...,False,False,False,False,False,False,False,False,False,False
12779,Proskill,NaN,NaN,19.0,PP,Bekasi,Private,Go,2001-04-27,zusmarrie27@gmail.com,...,False,False,False,False,False,False,False,False,False,False
2353,NaN,NaN,NaN,21.0,NST,"Kab. Banyuwangi, Jawa Timur",Private,Deluxe,2002-04-13,zuwihusna1304@gmail.com,...,False,False,False,False,False,False,False,True,True,True


# Tests and Assertions

In [75]:
# assert that all centers are mapped
center_in_df = df["Partner/Branch/Display Name"].unique()

unmapped = []
for c in center_in_df:
    if isinstance(c, str) and c.upper() not in module.center_map.keys():
        unmapped.append(c)

if len(unmapped) > 0:
    print(unmapped)
    raise Exception("There are unmapped centers. Map this inside modules/center_map.")

In [76]:
# assert that all membership codes has ben accounted
codes = df_clean["membership_code"].unique()
maps = pd.read_excel(Path("input/membership_mapping.xlsx"))["membership_code"].unique()

unmapped = []
for code in codes:
    if code not in maps:
        if code == np.NaN:
            continue
        unmapped.append(code)

if len(unmapped) > 0:
    for i in unmapped:
        print(i)
    raise Exception("Some membership are not mapped.")

# Save DF

In [77]:
import os
file= df_clean
path= f'output/member_data_{date}.parquet'

if not os.path.exists(path):
    file.to_parquet(path)
    print('File saved.')
else:
    print('File already exist.')

File saved.


/home/anj/miniconda3/envs/data-science/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


# Experiment

In [78]:
# df.loc[df['Membership/Display Name'].str.lower().str.contains('ilt', na= False)].sort_values('From')

In [79]:
# df['Membership/Display Name'].unique()

In [80]:
# df_clean['is_cpt'].value_counts()